<a href="https://colab.research.google.com/github/ricefan-tech/Transformer/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [5]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

In [ ]:
class MyEmbedding(nn.Module):
  def __init__(self, d_model, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, input):
    return self.embedding(input)

class MyPositionalEncoding(nn.Module):
  def __init__(self, seq_len, d_model):
    super().__init__()
    self.positionalEncoding = nn.Linear(seq_len, d_model, bias=False)

  def forward(self, input):
    return input + self.positionalEncoding

class MyMultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super().__init__()
    self.Q_proj = nn.Linear(d_model, d_model, bias=False)
    self.K_proj = nn.Linear(d_model, d_model, bias=False)
    self.V_proj = nn.Linear(d_model, d_model, bias=False)
    self.num_heads = num_heads

  def forward(self, q_input, v_input, k_input, padding_mask=None, causal_mask=None):
    Q = self.Q_matrix(q_input) # now has shape (batch_size, seq_len, d_model)
    K = self.K_proj(k_input)
    V = self.V_matrix(v_input)

    # before attn score calculation need to reshape into multi head
    batch_size, seq_len, d_model = Q.size()
    Q = Q.reshape(batch_size, seq_len, self.num_heads, d_model//self.num_heads).transpose(1, 2) #d_model must be divisible by num_heads
    K = K.reshape(batch_size, seq_len, self.num_heads, d_model//self.num_heads).transpose(1, 2)
    V = V.reshape(batch_size, seq_len, self.num_heads, d_model//self.num_heads).transpose(1, 2)

    if padding_mask:
      # padding mask is of shape (batch_size, seq_len), needs to be broadcasted to match Q@K.T which is (batch_size, num_head, seq_len, seq_len)
      padding_mask = padding_mask.unsqueeze(1).unsqueeze(2)
      attention_scores = Q @ K.transpose(-2, -1) / math.sqrt(d_model//self.num_heads)
      attention_scores = attention_scores.masked_fill(padding_mask, float("-inf"))

    else:
      attention_scors = Q @ K.transpose(-2, -1) / math.sqrt(d_model//self.num_heads)

    if causal_mask:
      # causal_mask is lower triangular matrix of shape target_seq_len, target_seq_len
      causal_mask = causal_mask.unsqueeze(1).unsqueeze(2)
      attention_scores = attention_scores.masked_fill(causal_mask, float("-inf"))

    attention_weights = torch.nn.softmax(attention_scores, dim=-1) @ V # is shape (batch_size, num_head, seq_len, head_dim)
    return attention_weights.reshape(batch_size, seq_len, d_model)


class FeedForwardNetwork(nn.Module):
  def __init__(self, d_model, ff_hidden):
    super().__init__()
    self.layer1 = nn.Linear(d_model, ff_hidden)
    self.layer2 = nn.Linear(ff_hidden, d_model)

  def forward(self, input):
    return self.layer2(nn.ReLU(self.layer1(input)))


class LayerNorm(nn.Module):
  def __init__(self, d_model, eps=1e-6):
    super().__init__()
    self.eps = eps #stabuliser for division

    self.gamma = nn.Parameter(torch.ones(d_model))
    self.beta = nn.Parameter(torch.zeros(d_model))

  def forward(self, input):
    mean = input.mean(dim=-1, keepdim=True)
    var = input.var(dim=-1, keepdim=True) #keeps broadcasted shape
    normalised_input = (input - mean)/ math.sqrt(var)
    return self.gamma * normalised_input + self.beta

In [4]:
attention_scores = torch.tensor([[[0.2, 0.4, 0.1],
                                  [0.5, 0.3, 0.2]]])
attention_scores.size()

torch.Size([1, 2, 3])